In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [8]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/chart/top/")

In [9]:
# Scroll till all movies load
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [10]:
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

In [11]:
rows = soup.find_all("li", class_="ipc-metadata-list-summary-item")

In [49]:
movie_name = []
movie_year = []
movie_duration = []
movie_avg_rating = []
movie_total_rating = []


page_links = []

for i in rows:
    # Movie Name
    name = i.find('h3').text
    movie_name.append(name)

    # Year of Release
    year_of_release = i.find_all('span',class_ = 'sc-15ac7568-7 cCsint cli-title-metadata-item')[0].text
    movie_year.append(year_of_release)
    
    # Duration
    duration = i.find_all('span',class_ = 'sc-15ac7568-7 cCsint cli-title-metadata-item')[1].text
    movie_duration.append(duration)
    
    # Average Rating
    avg_rating = i.find('span',class_ = 'ipc-rating-star--rating').text
    movie_avg_rating.append(avg_rating)
    
    # Total No of Rating
    total_rating = i.find('span',class_ = 'ipc-rating-star--voteCount').text
    movie_total_rating.append(total_rating)
    
    # Page Link for extracting Genre
    page_link = i.find("a", href=True)
    page_links.append(page_link["href"].split("?")[0])

In [47]:
base_url = "https://www.imdb.com" 

In [69]:
movie_genre = []

In [70]:
for i in page_links:
    link = base_url+i
    sub_resp = requests.get(link, headers=headers)
    sub_soup = BeautifulSoup(sub_resp.text, "html.parser")
    genre = sub_soup.find_all('a',class_ = 'ipc-chip ipc-chip--on-baseAlt')[-1].text
    movie_genre.append(genre)

In [71]:
len(movie_genre)

250

In [72]:
df = pd.DataFrame({
    'Name':movie_name,
    'Genre':movie_genre,
    'Year':movie_year,
    'Duration':movie_duration,
    'Average_Rating':movie_avg_rating,
    'Num_of_Rating':movie_total_rating
})

In [74]:
df.to_csv('IMDB_top250.csv')